# About

This development notebook is concerned with making sure that trip termination (see notebook 10) works.

In [1]:
import sys; sys.path.append("../src/")
from processing import fetch_archival_gtfs_realtime_data, parse_gtfs_into_action_log

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
gtfs_r = dict()

for n in range(0, 60, 5):
    print(n + 1)
    gtfs_r[n] = fetch_archival_gtfs_realtime_data(kind='gtfs', timestamp='2014-09-18-09-' + str(1 + n).zfill(2))
    
print("Done!")

1
6
11
16
21
26
31
36
41
46
51
56
Done!


We can use the same example we've been using up to now. In the following trip, the last stop is never actually stopped at.

In [4]:
gtfs_r[0].entity[0]

id: "000001"
trip_update {
  trip {
    trip_id: "047600_1..S02R"
    start_date: "20140918"
    route_id: "1"
  }
  stop_time_update {
    arrival {
      time: 1411044718
    }
    departure {
      time: 1411044838
    }
    stop_id: "137S"
  }
  stop_time_update {
    arrival {
      time: 1411044928
    }
    departure {
      time: 1411044928
    }
    stop_id: "138S"
  }
  stop_time_update {
    arrival {
      time: 1411045018
    }
    departure {
      time: 1411045078
    }
    stop_id: "139S"
  }
  stop_time_update {
    arrival {
      time: 1411045228
    }
    stop_id: "140S"
  }
}

In [13]:
gtfs_r[0].entity[1]

id: "000002"
vehicle {
  trip {
    trip_id: "047600_1..S02R"
    start_date: "20140918"
    route_id: "1"
  }
  current_stop_sequence: 35
  current_status: STOPPED_AT
  timestamp: 1411044718
  stop_id: "137S"
}

In [5]:
gtfs_r[5].entity[0]

id: "000001"
trip_update {
  trip {
    trip_id: "047600_1..S02R"
    start_date: "20140918"
    route_id: "1"
  }
  stop_time_update {
    arrival {
      time: 1411045639
    }
    stop_id: "140S"
  }
}

In [7]:
gtfs_r[5].entity[1]

id: "000002"
vehicle {
  trip {
    trip_id: "047600_1..S02R"
    start_date: "20140918"
    route_id: "1"
  }
  current_stop_sequence: 38
  current_status: IN_TRANSIT_TO
  timestamp: 1411045502
  stop_id: "140S"
}

In [6]:
gtfs_r[10].entity[0]

id: "000001"
trip_update {
  trip {
    trip_id: "048400_1..S02R"
    start_date: "20140918"
    route_id: "1"
  }
  stop_time_update {
    arrival {
      time: 1411045708
    }
    departure {
      time: 1411045768
    }
    stop_id: "139S"
  }
  stop_time_update {
    arrival {
      time: 1411045918
    }
    stop_id: "140S"
  }
}

To pick this up, we need to run these three feeds through simultaneously.

In [39]:
from processing import parse_feeds_into_trip_logs, mta_archival_time_to_unix_timestamp

In [9]:
ret3 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5], gtfs_r[10]], 
                                  [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'),
                                   mta_archival_time_to_unix_timestamp('2014-09-18-09-06'),
                                   mta_archival_time_to_unix_timestamp('2014-09-18-09-11')])

In [29]:
ret3["047600_1..S02R"]

,trip_id,route_id,action,stop_id,minimum_time,maximum_time,latest_information_time
0,047600_1..S02R,1,STOPPED_AT,137S,1.411045e+09,1.411045e+09,1.411045e+09
1,047600_1..S02R,1,STOPPED_OR_SKIPPED,138S,1.411045e+09,1.411046e+09,1.411046e+09
2,047600_1..S02R,1,STOPPED_OR_SKIPPED,139S,1.411045e+09,1.411046e+09,1.411046e+09
3,047600_1..S02R,1,STOPPED_OR_SKIPPED,140S,1.411046e+09,1.411046e+09,1.411046e+09


Perfect.

Compare this with the log we get when we do not finish the trip.

In [14]:
from processing import extract_synthetic_route_from_tripwise_action_logs, parse_message_into_action_log

In [23]:
extract_synthetic_route_from_tripwise_action_logs([
    parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1], 
                                  mta_archival_time_to_unix_timestamp('2014-09-18-09-01')),
    parse_message_into_action_log(gtfs_r[5].entity[0], gtfs_r[5].entity[1], 
                                  mta_archival_time_to_unix_timestamp('2014-09-18-09-06'))    
])

['137S', '138S', '139S', '140S']

In [34]:
from processing import parse_tripwise_action_logs_into_trip_log

In [48]:
parse_tripwise_action_logs_into_trip_log(
[
    parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1], 
                                  mta_archival_time_to_unix_timestamp('2014-09-18-09-01')),
    parse_message_into_action_log(gtfs_r[5].entity[0], gtfs_r[5].entity[1], 
                                  mta_archival_time_to_unix_timestamp('2014-09-18-09-06'))
   
])

,trip_id,route_id,action,stop_id,minimum_time,maximum_time,latest_information_time
0,047600_1..S02R,1,STOPPED_AT,137S,1.411045e+09,1.411045e+09,1.411045e+09
1,047600_1..S02R,1,STOPPED_OR_SKIPPED,138S,1.411045e+09,1.411046e+09,1.411046e+09
2,047600_1..S02R,1,STOPPED_OR_SKIPPED,139S,1.411045e+09,1.411046e+09,1.411046e+09
3,047600_1..S02R,1,EN_ROUTE_TO,140S,1.411046e+09,NaN,1.411046e+09


We can finish this trip using the `finish_trip` function (which is used internally within the whole-feeds parser):

In [49]:
from processing import finish_trip
finish_trip(_, mta_archival_time_to_unix_timestamp('2014-09-18-09-11'))

,trip_id,route_id,action,stop_id,minimum_time,maximum_time,latest_information_time
0,047600_1..S02R,1,STOPPED_AT,137S,1.411045e+09,1.411045e+09,1.411045e+09
1,047600_1..S02R,1,STOPPED_OR_SKIPPED,138S,1.411045e+09,1.411046e+09,1.411046e+09
2,047600_1..S02R,1,STOPPED_OR_SKIPPED,139S,1.411045e+09,1.411046e+09,1.411046e+09
3,047600_1..S02R,1,STOPPED_OR_SKIPPED,140S,1.411046e+09,1.411046e+09,1.411046e+09


In [40]:
ret3 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5], gtfs_r[10]], 
                                  [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'),
                                   mta_archival_time_to_unix_timestamp('2014-09-18-09-06'),
                                   mta_archival_time_to_unix_timestamp('2014-09-18-09-11')])

In [41]:
ret3["047600_1..S02R"]

,trip_id,route_id,action,stop_id,minimum_time,maximum_time,latest_information_time
0,047600_1..S02R,1,STOPPED_AT,137S,1.411045e+09,1.411045e+09,1.411045e+09
1,047600_1..S02R,1,STOPPED_OR_SKIPPED,138S,1.411045e+09,1.411046e+09,1.411046e+09
2,047600_1..S02R,1,STOPPED_OR_SKIPPED,139S,1.411045e+09,1.411046e+09,1.411046e+09
3,047600_1..S02R,1,STOPPED_OR_SKIPPED,140S,1.411046e+09,1.411046e+09,1.411046e+09
